In [167]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum as Sum
from pyspark.sql.types import ArrayType,StringType,FloatType
from pyspark.sql.functions import col, udf, explode
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, HBox, widgets
import pgeocode
from haversine import haversine, Unit

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName('MilestoneOneProject') \
    .getOrCreate() 

sc = spark.sparkContext

In [168]:
# # read csv
# master_data = spark.read.csv('../data_combined/master_data.csv', header=True)

AnalysisException: Path does not exist: file:/Users/exb002y/personalProjects/MilestoneOneProject/data_combined/master_data.csv;

In [169]:
# create view
# master_data.createOrReplaceTempView("master_data")


In [170]:
# # query the dataset
# query = """
#     SELECT *
#     FROM master_data
#     LIMIT 50
# """
# # spark.sql(query).show()

In [171]:
lat_long = ''
def get_lat_long(zip_):
    # using a library called pgeocode
    nomi = pgeocode.Nominatim('US')
    geoinfo = nomi.query_postal_code(zip_)
    lat = geoinfo.loc['latitude']
    long = geoinfo.loc['longitude']
    lat_long = lat, long
    print(lat_long)

def callback(wdgt):
    lat_long = get_lat_long(wdgt.value)
    print(lat_long)
    

text=widgets.Text(
    value='',
    placeholder='30082',
    description='Zip:',
    disabled=False
)

display(text)
text.on_submit(callback)

Text(value='', description='Zip:', placeholder='30082')

In [172]:
def haversine_(lat,long):
    user = (33.8631, -84.5382) # (lat, lon)
    ingred = (float(lat),float(long))
    distance = haversine(user, ingred)
    return distance
    
spark.udf.register('haversine_', haversine_, FloatType())

<function __main__.haversine_(lat, long)>

In [173]:
haversine_apply_query = """
    SELECT lat, long, raw_ingredient, title, recipe_ingredients, haversine_(lat, long) as distance
    FROM master_data
    WHERE lat != 0 AND long !=0
"""
distance = spark.sql(haversine_apply_query)
dist_table = distance.createOrReplaceTempView("dist_table")
# distance.show()

In [174]:
%%timeit -n 1 -r 1
# distance.show()

ValueError: empty body on For

In [166]:
statement = """
    SELECT title, distance
    FROM dist_table 
    WHERE title = "Boudin Blanc Terrine with Red Onion Confit "
"""
spark.sql(statement).show()

# groupby_min = """
#     SELECT title, recipe_ingredients, MIN(distance) as dist
#     FROM dist_table
#     GROUP BY title, recipe_ingredients
#     ORDER BY dist asc
# """
# spark.sql(groupby_min).show()

Py4JJavaError: An error occurred while calling o1378.showString.
: org.apache.spark.SparkException: Job 134 cancelled 
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:1989)
	at org.apache.spark.scheduler.DAGScheduler.handleJobCancellation(DAGScheduler.scala:1924)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:2172)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:2155)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:2144)
	at org.apache.spark.util.EventLoop$$anon$1.run(EventLoop.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.runJob(DAGScheduler.scala:758)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2116)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2137)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2156)
	at org.apache.spark.sql.execution.SparkPlan.executeTake(SparkPla